In [ ]:
import os
import time
import math
from multiprocessing import Pool, cpu_count

import laspy
import numpy as np
import pandas as pd
from numba import jit
from numba.core import types
from numba.typed import Dict

In [ ]:
# --- CONFIGURATION ---
# Set all simulation parameters in this section.

# 1. Input Data and Scene Definition
LIDAR_FILE_PATH = 'data/houselas_re_veg_2.las'
OUTPUT_DIRECTORY = 'results/shadow_matrix_results'
OUTPUT_FILENAME = 'shadow_attenuation_matrix_conecasting.csv'
BOUNDING_BOX = None

# ASPRS Standard Classification Codes
# Including multiple vegetation classes (3: Low, 4: Medium, 5: High)
RELEVANT_CLASSES = {2, 3, 4, 5, 6}
GROUND_CLASS_CODE = 2
BUILDING_CLASS_CODE = 6
VEGETATION_CLASS_CODES = {3, 4, 5}

# Priority for voxel classification (higher number = higher priority)
# High vegetation (5) has priority over medium (4), etc.
CLASS_PRIORITY = {6: 4, 5: 3, 4: 2, 3: 1, 2: 0, 0: -1}

# 2. Voxelization Parameters
VOXEL_SIZE = 0.5

# 3. Solar Position & Cone-Casting Simulation Parameters
AZIMUTH_STEPS = 360  # 1-degree steps
ELEVATION_STEPS = 91 # 1-degree steps (0-90 inclusive)

# --- NEW: Cone-Casting Parameters ---
# The sun's angular radius is approx 0.265 degrees.
SOLAR_ANGULAR_RADIUS_DEG = 0.265
# Number of rays to cast to approximate the solar disk. More rays = more
# accurate penumbra but longer computation time.
NUM_RAYS_PER_CONE = 16

# 4. Ray-Casting and Attenuation Parameters
# --- UPDATED: Class-specific extinction coefficients ---
# Assign a different base extinction coefficient (k) to each vegetation class.
# These values may need to be calibrated for specific vegetation types.
VEGETATION_EXTINCTION_COEFFICIENTS = {
    3: 0.7,  # k for Low Vegetation
    4: 0.6,  # k for Medium Vegetation
    5: 0.4   # k for High Vegetation
}

# 5. Performance Parameters
NUM_CORES = cpu_count()

In [ ]:
# --- MODULE 1: DATA INGESTOR & PREPARER ---

def load_and_prepare_lidar(file_path, bounding_box=None, relevant_classes=None):
    """
    Reads a LAS/LAZ file, filters points by classification and bounding box.
    """
    print(f"Loading LiDAR data from {file_path}...")
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return None, None
    try:
        las = laspy.read(file_path)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None, None

    points_xyz = np.vstack((las.x, las.y, las.z)).transpose()
    classifications = np.array(las.classification)

    mask = np.ones(len(points_xyz), dtype=bool)
    if bounding_box:
        mask &= (
            (points_xyz[:, 0] >= bounding_box['X_MIN']) & (points_xyz[:, 0] < bounding_box['X_MAX']) &
            (points_xyz[:, 1] >= bounding_box['Y_MIN']) & (points_xyz[:, 1] < bounding_box['Y_MAX'])
        )
    if relevant_classes:
        mask &= np.isin(classifications, list(relevant_classes))

    filtered_points = points_xyz[mask]
    filtered_classifications = classifications[mask]

    if len(filtered_points) == 0:
        print("Warning: No points remaining after filtering.")
        return None, None
    print(f"Data loaded and filtered. {len(filtered_points)} points remaining.")
    return filtered_points, filtered_classifications

In [ ]:
# --- MODULE 2: VOXELIZER ---

def voxelize_scene(points, classifications, voxel_size):
    """
    Converts a point cloud into classification and density voxel grids.
    """
    if points is None or len(points) == 0:
        return None, None, None, None

    print("Voxelizing the scene...")
    scene_min = np.min(points, axis=0)
    scene_max = np.max(points, axis=0)
    grid_dims = np.ceil((scene_max - scene_min) / voxel_size).astype(int)
    print(f"Voxel grid dimensions: {grid_dims}")

    classification_grid = np.zeros(grid_dims, dtype=np.int8)
    density_grid = np.zeros(grid_dims, dtype=np.float32)

    voxel_indices = np.floor((points - scene_min) / voxel_size).astype(int)
    for i in range(3):
        voxel_indices[:, i] = np.clip(voxel_indices[:, i], 0, grid_dims[i] - 1)

    print("Populating classification and density grids...")
    for i in range(len(points)):
        idx = tuple(voxel_indices[i])
        current_class = classifications[i]
        if CLASS_PRIORITY.get(current_class, -1) > CLASS_PRIORITY.get(classification_grid[idx], -1):
            classification_grid[idx] = current_class
        if current_class in VEGETATION_CLASS_CODES:
            density_grid[idx] += 1

    voxel_volume = voxel_size ** 3
    vegetation_voxels = np.isin(classification_grid, list(VEGETATION_CLASS_CODES))
    density_grid[vegetation_voxels] /= voxel_volume
    
    print("Voxelization complete.")
    return classification_grid, density_grid, scene_min, grid_dims

In [ ]:
# --- MODULE 3: RAY-CASTING & CONE-CASTING ENGINE ---

def generate_cone_vectors(center_direction, radius_rad, num_samples):
    """
    Generates a set of vectors distributed within a cone around a center direction.
    """
    # Create a basis (a coordinate system) aligned with the center_direction
    if np.allclose(np.abs(center_direction), [0, 0, 1]):
        # Handle case where direction is along Z-axis
        v_up = np.array([0, 1, 0])
    else:
        v_up = np.array([0, 0, 1])
    
    u = np.cross(v_up, center_direction)
    u /= np.linalg.norm(u)
    v = np.cross(center_direction, u)

    cone_vectors = []
    # Use stratified sampling for better distribution
    for i in range(num_samples):
        # Sample radius and angle to get points on a disk
        r = radius_rad * np.sqrt((i + 0.5) / num_samples)
        theta = 2 * np.pi * 0.61803398875 * i # Golden angle for good distribution

        # Map disk point to 3D offset and add to center direction
        offset_vec = r * (np.cos(theta) * u + np.sin(theta) * v)
        new_vec = center_direction + offset_vec
        new_vec /= np.linalg.norm(new_vec) # Re-normalize
        cone_vectors.append(new_vec)
        
    return cone_vectors

@jit(nopython=True)
def trace_ray_fast(ray_origin, ray_direction, scene_min, voxel_size, grid_dims):
    """
    An efficient voxel traversal algorithm (Amanatides-Woo).
    """
    ray_pos = (ray_origin - scene_min) / voxel_size
    ix, iy, iz = int(ray_pos[0]), int(ray_pos[1]), int(ray_pos[2])
    step_x = 1 if ray_direction[0] >= 0 else -1
    step_y = 1 if ray_direction[1] >= 0 else -1
    step_z = 1 if ray_direction[2] >= 0 else -1
    
    next_voxel_boundary_x = (ix + (step_x > 0)) * voxel_size + scene_min[0]
    next_voxel_boundary_y = (iy + (step_y > 0)) * voxel_size + scene_min[1]
    next_voxel_boundary_z = (iz + (step_z > 0)) * voxel_size + scene_min[2]
    
    t_max_x = (next_voxel_boundary_x - ray_origin[0]) / ray_direction[0] if ray_direction[0] != 0 else np.inf
    t_max_y = (next_voxel_boundary_y - ray_origin[1]) / ray_direction[1] if ray_direction[1] != 0 else np.inf
    t_max_z = (next_voxel_boundary_z - ray_origin[2]) / ray_direction[2] if ray_direction[2] != 0 else np.inf
    
    t_delta_x = voxel_size / abs(ray_direction[0]) if ray_direction[0] != 0 else np.inf
    t_delta_y = voxel_size / abs(ray_direction[1]) if ray_direction[1] != 0 else np.inf
    t_delta_z = voxel_size / abs(ray_direction[2]) if ray_direction[2] != 0 else np.inf

    while True:
        if not (0 <= ix < grid_dims[0] and 0 <= iy < grid_dims[1] and 0 <= iz < grid_dims[2]):
            break
        yield (ix, iy, iz)
        if t_max_x < t_max_y:
            if t_max_x < t_max_z:
                ix += step_x; t_max_x += t_delta_x
            else:
                iz += step_z; t_max_z += t_delta_z
        else:
            if t_max_y < t_max_z:
                iy += step_y; t_max_y += t_delta_y
            else:
                iz += step_z; t_max_z += t_delta_z

@jit(nopython=True)
def calculate_transmittance(voxel_path_generator, classification_grid, density_grid, voxel_size, k_coeffs):
    """
    Calculates the final transmittance for a single ray path, using class-specific
    extinction coefficients for different vegetation types.
    """
    transmittance = 1.0
    path_length_in_voxel = voxel_size
    for ix, iy, iz in voxel_path_generator:
        voxel_class = classification_grid[ix, iy, iz]
        if voxel_class == BUILDING_CLASS_CODE: return 0.0
        
        # Check if the voxel is any type of vegetation
        if voxel_class in (3, 4, 5):
            k_base = k_coeffs.get(voxel_class, 0.0) # Look up the correct k
            if k_base > 0:
                density = density_grid[ix, iy, iz]
                if density > 0:
                    k = k_base * density
                    transmittance *= math.exp(-k * path_length_in_voxel)
        
        if transmittance < 1e-6: return 0.0
    return transmittance

def run_single_ray_simulation(args):
    """
    Worker function to trace a single ray. For use with multiprocessing.
    """
    ray_direction, analysis_point, grid_dims, sim_data = args
    classification_grid = sim_data['classification_grid']
    density_grid = sim_data['density_grid']
    voxel_size = sim_data['voxel_size']
    k_coeffs = sim_data['k_coeffs']
    scene_min = sim_data['scene_min']
    
    voxel_path_generator = trace_ray_fast(analysis_point, ray_direction, scene_min, voxel_size, grid_dims)
    return calculate_transmittance(voxel_path_generator, classification_grid, density_grid, voxel_size, k_coeffs)

In [ ]:
# --- MAIN EXECUTION ---
def run_all():
# if __name__ == '__main__':
    start_time = time.time()
    
    # 1. Load and Prepare Data
    points, classifications = load_and_prepare_lidar(
        LIDAR_FILE_PATH, BOUNDING_BOX, RELEVANT_CLASSES
    )
    if points is None: exit()

    # 2. Voxelize Scene
    classification_grid, density_grid, scene_min, grid_dims = voxelize_scene(
        points, classifications, VOXEL_SIZE
    )
    if classification_grid is None: exit()
    scene_max = scene_min + grid_dims * VOXEL_SIZE

    # 3. Define the single analysis point at the center of the scene
    scene_center_x = (scene_min[0] + scene_max[0]) / 2
    scene_center_y = (scene_min[1] + scene_max[1]) / 2

    ground_points_all = points[classifications == GROUND_CLASS_CODE]
    if len(ground_points_all) > 0:
        search_radius = 5.0
        center_points_mask = (
            (ground_points_all[:, 0] > scene_center_x - search_radius) &
            (ground_points_all[:, 0] < scene_center_x + search_radius) &
            (ground_points_all[:, 1] > scene_center_y - search_radius) &
            (ground_points_all[:, 1] < scene_center_y + search_radius)
        )
        ground_points_near_center = ground_points_all[center_points_mask]
        if len(ground_points_near_center) > 0:
            ground_z = np.min(ground_points_near_center[:, 2]) + 0.01
        else:
            print("Warning: No ground points near center. Using scene minimum Z.")
            ground_z = scene_min[2] + 0.01
    else:
        print("Warning: No ground points in dataset. Using scene minimum Z.")
        ground_z = scene_min[2] + 0.01

    analysis_point = np.array([scene_center_x, scene_center_y, ground_z])
    print(f"Analysis point set to scene center: {analysis_point}")
    
    # # Prepare a Numba-compatible dictionary for the JIT-compiled function
    # k_coeffs_numba = Dict.empty(key_type=types.int64, value_type=types.float64)
    # for k, v in VEGETATION_EXTINCTION_COEFFICIENTS.items():
    #     k_coeffs_numba[k] = v

    # simulation_data = {
    #     'classification_grid': classification_grid,
    #     'density_grid': density_grid,
    #     'voxel_size': VOXEL_SIZE,
    #     'k_coeffs': k_coeffs_numba,
    #     'scene_min': scene_min
    # }

    # --- FIX: Pass the standard python dictionary ---
    simulation_data = {
        'classification_grid': classification_grid,
        'density_grid': density_grid,
        'voxel_size': VOXEL_SIZE,
        'k_coeffs': VEGETATION_EXTINCTION_COEFFICIENTS, # Pass the standard dict
        'scene_min': scene_min
    }

    # 4. Define Solar Angles and Run Simulation Loop
    azimuths = np.linspace(0, 2 * np.pi, AZIMUTH_STEPS, endpoint=False)
    elevations = np.linspace(0, np.pi / 2, ELEVATION_STEPS, endpoint=True)
    solar_radius_rad = np.deg2rad(SOLAR_ANGULAR_RADIUS_DEG)
    
    simulation_results = []
    
    total_directions = len(azimuths) * (len(elevations) - 1)
    print(f"\n--- Starting Cone-Casting Simulation for Center Point ---")
    print(f"Casting {NUM_RAYS_PER_CONE} rays per cone for each of {total_directions} solar positions...")
    
    current_direction = 0
    with Pool(processes=NUM_CORES) as pool:
        for el in elevations:
            if el < 0.001: continue
            
            for az in azimuths:
                current_direction += 1
                if current_direction % 500 == 0:
                     print(f"Processing direction {current_direction} of {total_directions} (Az: {np.rad2deg(az):.1f}°, El: {np.rad2deg(el):.1f}°)...")
                
                center_ray_direction = np.array([np.cos(el) * np.sin(az), np.cos(el) * np.cos(az), np.sin(el)])
                
                cone_ray_vectors = generate_cone_vectors(center_ray_direction, solar_radius_rad, NUM_RAYS_PER_CONE)
                
                task_args = [(vec, analysis_point, grid_dims, simulation_data) for vec in cone_ray_vectors]
                
                transmittances = pool.map(run_single_ray_simulation, task_args)
                
                avg_transmittance = np.mean(transmittances)
                
                simulation_results.append({
                    'azimuth': az, 'elevation': el, 'transmittance': avg_transmittance
                })

    # 5. Format and Save Final Attenuation Matrix
    print("\n--- Aggregating Results into CSV Matrix ---")
    if not simulation_results:
        print("No results to save. Exiting.")
        exit()

    df = pd.DataFrame(simulation_results)
    df['azimuth_deg'] = np.round(np.rad2deg(df['azimuth'])).astype(int)
    df['elevation_deg'] = np.round(np.rad2deg(df['elevation'])).astype(int)
    
    shadow_matrix_df = df.pivot_table(
        index='elevation_deg', columns='azimuth_deg', values='transmittance'
    )
    
    shadow_matrix_df.index = [f"Altitude_{i}" for i in shadow_matrix_df.index]
    shadow_matrix_df.columns = [f"Azimuth_{c}" for c in shadow_matrix_df.columns]
    
    shadow_matrix_df = shadow_matrix_df.sort_index(key=lambda x: np.array([int(i.split('_')[1]) for i in x]))
    shadow_matrix_df = shadow_matrix_df.sort_index(axis=1, key=lambda x: np.array([int(i.split('_')[1]) for i in x]))

    if not os.path.exists(OUTPUT_DIRECTORY):
        os.makedirs(OUTPUT_DIRECTORY)
        
    output_path = os.path.join(OUTPUT_DIRECTORY, OUTPUT_FILENAME)
    shadow_matrix_df.to_csv(output_path)
    
    end_time = time.time()
    print(f"\n--- Simulation Finished ---")
    print(f"Total execution time: {end_time - start_time:.2f} seconds")
    print(f"Saved shadow attenuation matrix to {output_path}")

In [ ]:
run_all()